In [36]:
import pandas as pd
import numpy as np
import r2pipe as r2
from tqdm import tqdm

BYTE_LENGTH = 2000
N_GRAM_1 = 2
N_GRAM_2 = 4
N_WAYS = 10
SEED = 7
NUM_EXAMPLES = 100
NUM_EXAMPLES_TEST = 30
NUM_EXAMPLES_VAL = 0

# load dataset
DATASET_FOLDER = "/home/mandy900619/data/Malware202403/"
CLUSTER_PATH = "./cluster_data/"
CPU_ARCH = "i386"
DATASET_PATH = f"./dataset/malware_original_{CPU_ARCH}.csv"  
EMBEDDING_PATH = "./dataset_embedding/" 
ADDITONAL_INFO = "_rm_dup"

print(f"Loading {CPU_ARCH} dataset from {DATASET_PATH}...")
dataset = pd.read_csv(DATASET_PATH)


Loading i386 dataset from ./dataset/malware_original_i386.csv...


In [37]:
family_counts = dataset['family'].value_counts()
print(family_counts)

family
xorddos        906935
mirai           36877
gafgyt          28643
setag            2551
tsunami          1999
dofloo           1064
ddostf            683
meterpreter       313
elknot            266
chinaz            224
kaiji             130
dnsamp             94
mayday             83
race               62
exploitscan        55
sshbrute           55
Name: count, dtype: int64


In [38]:
# extract byte sequences from ELF files

notHaveByteSequence = False
removeDup = False

def split_hex_string(hex_string):
    return " ".join([hex_string[i:i+2] for i in range(0, len(hex_string), 2)])

if notHaveByteSequence:
    # extract byte sequences
    print(f"Extract byte sequences from {CPU_ARCH} dataset...")
    print(f"Extracting byte sequences of length {BYTE_LENGTH}...")

    for row in tqdm(dataset.itertuples(), total=len(dataset)):
        # open file with r2
        byteAnalysis = r2.open(DATASET_FOLDER + row.file_name[:2] + "/" + row.file_name, flags=["-2"])
        out = byteAnalysis.cmd(f"px* {BYTE_LENGTH}")
        byteAnalysis.cmd("quit")
        lines = out.strip().split("\n")
        byteSeqence = [line[3:-1] for line in lines if not line.startswith("s-")]
        byteSeqence = "".join(byteSeqence)
        byteSeqence = split_hex_string(byteSeqence)
        dataset.at[row.Index, "byte_sequence"] = byteSeqence

In [39]:
# output dataset
if notHaveByteSequence:  
    OUTPUT_PATH = f"./dataset/malware_original_{CPU_ARCH}_byte_sequence{BYTE_LENGTH}_split.csv"
    dataset.to_csv(OUTPUT_PATH, index=False)


In [40]:
if removeDup:
    dataset = pd.read_csv(f"./dataset/malware_original_{CPU_ARCH}_byte_sequence{BYTE_LENGTH}_split.csv")
    # remove duplicate rows based on byte_sequence
    print("Original dataset shape:", dataset.shape)
    print("Removing duplicate rows based on byte_sequence...")
    dataset_rm_dup = dataset.drop_duplicates(subset="byte_sequence")
    dataset_rm_dup = dataset_rm_dup.reset_index(drop=True)
    print("Dataset shape after removing duplicates:", dataset_rm_dup.shape)
    family_counts = dataset_rm_dup["family"].value_counts()
    print(family_counts[:])

    # output dataset
    OUTPUT_PATH = f"./dataset/malware_original_{CPU_ARCH}_byte_sequence{BYTE_LENGTH}_split{ADDITONAL_INFO}.csv"
    dataset_rm_dup.to_csv(OUTPUT_PATH, index=False)

In [41]:
dataset = pd.read_csv(f"./dataset/malware_original_{CPU_ARCH}_byte_sequence{BYTE_LENGTH}_split{ADDITONAL_INFO}.csv")
family_counts = dataset["family"].value_counts()
print(family_counts[:])

family
mirai           14098
gafgyt           7927
tsunami           679
race               51
exploitscan        47
xorddos            41
sshdoor            33
local              32
rkit               31
sshbrute           30
kaiji              28
sliver             27
backegmm           27
dnsamp             27
pnscan             26
meterpreter        26
prochider          26
cornelgen          24
equationdrug       21
sckit              20
chinaz             17
ddostf             15
setag              15
elknot             12
dofloo              7
mayday              6
Name: count, dtype: int64


In [9]:
family = dataset['family'].value_counts()[:(N_WAYS)].index
dataset_exp = dataset[dataset['family'].isin(family)]

print(dataset_exp.shape)


dataset_train = dataset_exp.groupby('family').sample(n=NUM_EXAMPLES, random_state=SEED)

dataset_test = dataset_exp[~dataset_exp.index.isin(dataset_train.index)].groupby('family').sample(n=NUM_EXAMPLES_TEST, random_state=SEED)
if NUM_EXAMPLES_VAL > 0:
    dataset_val = dataset_exp[~dataset_exp.index.isin(dataset_train.index) & ~dataset_exp.index.isin(dataset_test.index)].groupby('family').sample(n=NUM_EXAMPLES_VAL, random_state=SEED)

print(dataset_train.shape)
print(dataset_test.shape)
if NUM_EXAMPLES_VAL > 0:
    print(dataset_val.shape)


byteSeqenceTrain = dataset_train['byte_sequence'].values
byteSeqenceTest = dataset_test['byte_sequence'].values
if NUM_EXAMPLES_VAL > 0:
    byteSeqenceVal = dataset_val['byte_sequence'].values
y_train = dataset_train['family'].values
y_test = dataset_test['family'].values
if NUM_EXAMPLES_VAL > 0:
    y_val = dataset_val['family'].values


# convert y family to number
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
if NUM_EXAMPLES_VAL > 0:
    list_ = list(y_train) + list(y_test) + list(y_val)
else:
    list_ = list(y_train) + list(y_test)
le.fit(list_)
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
if NUM_EXAMPLES_VAL > 0:
    y_val = le.fit_transform(y_val)


(33994, 14)
(1000, 14)
(300, 14)


In [11]:
# extract tf-idf features
from sklearn.feature_extraction.text import TfidfVectorizer
# 4-grams
tfidf_vec = TfidfVectorizer(analyzer='word', ngram_range=(N_GRAM_1, N_GRAM_2), max_features=1000) # , max_features=1000
tfidf_matrix_train = tfidf_vec.fit_transform(byteSeqenceTrain)
tfidf_matrix_test = tfidf_vec.transform(byteSeqenceTest)
if NUM_EXAMPLES_VAL > 0:
    tfidf_matrix_val = tfidf_vec.transform(byteSeqenceVal)

tfidf_matrix_train = tfidf_matrix_train.toarray()
tfidf_matrix_test = tfidf_matrix_test.toarray()
if NUM_EXAMPLES_VAL > 0:
    tfidf_matrix_val = tfidf_matrix_val.toarray()

label_mapping = {index: label for index, label in enumerate(le.classes_)}

In [13]:
print(f"Training set shape: {tfidf_matrix_train.shape}")
print(f"Testing set shape: {tfidf_matrix_test.shape}")
if NUM_EXAMPLES_VAL > 0:
    print(f"Validation set shape: {tfidf_matrix_val.shape}")
print(f"Label mapping: {label_mapping}")

Training set shape: (1000, 1000)
Testing set shape: (300, 1000)
Label mapping: {0: 'camelot', 1: 'dropperl', 2: 'gafgyt', 3: 'mirai', 4: 'rekoobe', 5: 'sliver', 6: 'sshdoor', 7: 'tsunami', 8: 'vtflooder', 9: 'xmrig'}


In [14]:
import pickle

with open(f"{EMBEDDING_PATH}{CPU_ARCH}_label_mapping{ADDITONAL_INFO}.pickle", 'wb') as f:
    pickle.dump(label_mapping, f)
    f.close()
with open(f"{EMBEDDING_PATH}{CPU_ARCH}_tfidf_vec_train{ADDITONAL_INFO}.pickle", 'wb') as f:
    pickle.dump(tfidf_matrix_train, f)
    f.close()
with open(f"{EMBEDDING_PATH}{CPU_ARCH}_tfidf_vec_test{ADDITONAL_INFO}.pickle", 'wb') as f:
    pickle.dump(tfidf_matrix_test, f)
    f.close()
if NUM_EXAMPLES_VAL > 0:
    with open(f"{EMBEDDING_PATH}{CPU_ARCH}_tfidf_vec_val{ADDITONAL_INFO}.pickle", 'wb') as f:
        pickle.dump(tfidf_matrix_val, f)
        f.close()
with open(f"{EMBEDDING_PATH}{CPU_ARCH}_y_train{ADDITONAL_INFO}.pickle", 'wb') as f:
    pickle.dump(y_train, f)
    f.close()
with open(f"{EMBEDDING_PATH}{CPU_ARCH}_y_test{ADDITONAL_INFO}.pickle", 'wb') as f:
    pickle.dump(y_test, f)
    f.close()
if NUM_EXAMPLES_VAL > 0:
    with open(f"{EMBEDDING_PATH}{CPU_ARCH}_y_val{ADDITONAL_INFO}.pickle", 'wb') as f:
        pickle.dump(y_val, f)
        f.close()

In [15]:
# Format is a vector one line, each dimension value split by blank space
# cluster preprocessing
for key in label_mapping:
    outputPathTrain = f"{CLUSTER_PATH}{CPU_ARCH}_{label_mapping[key]}_train{ADDITONAL_INFO}.txt"
    # outputPathTest = f"{CLUSTER_PATH}{CPU_ARCH}_{label_mapping[key]}_test.txt"
    with open(outputPathTrain, 'w') as f:
        for i in range(len(tfidf_matrix_train)):
            if y_train[i] == key:
                f.write('\t'.join(map(str, tfidf_matrix_train[i])) + "\n")
    f.close()
    # with open(outputPathTest, 'w') as f:
    #     for i in range(len(tfidf_matrix_test)):
    #         if y_test[i] == key:
    #             f.write('\t'.join(map(str, tfidf_matrix_test[i])) + "\n")